# Download dataset

In [ ]:
!pip install SoccerNet

In [ ]:
from SoccerNet.Downloader import SoccerNetDownloader as SNdl
mySNdl = SNdl(LocalDirectory="./data")
mySNdl.downloadDataTask(task="jersey-2023", split=["train","test","challenge"])

In [4]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch

In [11]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
train_dataset = datasets.ImageFolder(
        "testing/tiny-imagenet-200",
        transforms.Compose([
            transforms.RandomResizedCrop(32),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=64, shuffle=True,
        num_workers=1, pin_memory=True)

In [12]:
a = next(iter(train_loader))   
a[0].shape, a[1].shape

(torch.Size([64, 3, 32, 32]), torch.Size([64]))

# Testing